In [8]:
import torch
import os

In [9]:
def save_torch_tensor(tensor, filename='tensor.pt'):
    save_dir = os.path.expanduser('~/Downloads/saved_tensors')
    os.makedirs(save_dir, exist_ok=True)
    file_path = os.path.join(save_dir, filename)
    torch.save(tensor, file_path)
    print(f"Saved tensor to {file_path}")
def load_torch_tensor(filename='tensor.pt'):
    load_path = os.path.join(os.path.expanduser('~/Downloads/saved_tensors'), filename)
    if not os.path.exists(load_path):
        raise FileNotFoundError(f"No tensor file found at {load_path}")
    tensor = torch.load(load_path, map_location=torch.device('cpu'))
    print(f"Loaded tensor from {load_path}")
    return tensor

In [10]:
target_binned = load_torch_tensor()

Loaded tensor from /Users/jan/Downloads/saved_tensors/tensor.pt


In [17]:
import torch
import torch.nn.functional as F

def dlbd(input_tensor, kernel_size=5, sigma=1.0):
    """
    input_tensor shape: (B, C, H, W)
    kernel_size: size of the gaussian kernel (must be odd)
    sigma: standard deviation of the gaussian
    """
    # 1D gaussian
    coords = torch.arange(kernel_size) - kernel_size // 2
    g_1d = torch.exp(-(coords**2) / (2 * sigma**2))
    g_1d /= g_1d.sum()  # normalize

    # 2D gaussian kernel
    g_2d = g_1d[:, None] * g_1d[None, :]
    g_2d = g_2d.view(1, 1, kernel_size, kernel_size)
    g_2d = g_2d.to(input_tensor.device, dtype=input_tensor.dtype)

    # Repeat kernel for every channel
    # -> shape (C, 1, K, K) so each channel is convolved separately
    g_2d = g_2d.repeat(input_tensor.shape[1], 1, 1, 1)
    print(g_2d.shape)
    print(f'Input tensor shape {input_tensor.shape}')

    # Pad so output size matches input size
    padding = kernel_size // 2

    # Depthwise convolution
    return F.conv2d(
        input_tensor, 
        g_2d, 
        groups=input_tensor.shape[1], 
        padding=padding
    )

In [18]:
blurred = dlbd(target_binned)

torch.Size([32, 1, 5, 5])
Input tensor shape torch.Size([4, 32, 256, 256])
